# Space X Falcon 9 First Stage Landing Prediction
### **Machine Learning Prediction**
![Falcon 9](../images/falcon9.webp)
### Objective:
* Perform Exploratory Data Analysis (EDA)
* Determine Training Labels.

# Author 
[Helena Pedro](https://www.linkedin.com/in/helena-mbeua-pedro/) is a Data Scientist at Millennium Atlantic Bank in Angola. She is a Creative big thinker passionated about using data and optimization tools to direct decision making and solve complex and large-scale challenges.
- **Email:** mbeua94@gmail.com


© 2024 